In [3]:
# Install necessary libraries
!pip install pytorch-tabnet
!pip install captum
!pip install optuna
!pip install imbalanced-learn
!pip install dask-expr
!pip install scikit-learn-contrib
!pip install lightgbm

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and modeling
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Handling imbalanced data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

# Deep Learning Model
from pytorch_tabnet.tab_model import TabNetClassifier

# Explainable AI
import shap
from captum.attr import IntegratedGradients

# Hyperparameter Optimization
import optuna
from optuna import Trial

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# For model saving and loading
import joblib

# Import torch for TabNet
import torch

# Define the PermutationImportanceTabNet class
class PermutationImportanceTabNet(TabNetClassifier):
    def __init__(self, input_dim, feature_names, permutation_prob=0.1, importance_decay=0.99, *args, **kwargs):
        """
        Initializes the PermutationImportanceTabNet.

        Parameters:
        - input_dim (int): Number of input features.
        - feature_names (list): List of feature names.
        - permutation_prob (float): Probability of applying permutation during a forward pass.
        - importance_decay (float): Decay factor for importance scores to smooth over epochs.
        - *args, **kwargs: Additional arguments for TabNetClassifier.
        """
        super(PermutationImportanceTabNet, self).__init__(input_dim=input_dim, *args, **kwargs)
        self.permutation_prob = permutation_prob
        self.importance_scores = torch.zeros(input_dim)
        self.importance_decay = importance_decay  # To smooth importance scores
        self.feature_names = feature_names  # List of feature names for interpretability

    def forward(self, X, y=None):
        """
        Overrides the forward pass to include permutation-based feature importance.

        Parameters:
        - X (torch.Tensor): Input features.
        - y (torch.Tensor, optional): Target labels.

        Returns:
        - out (torch.Tensor): Model outputs.
        - M_loss (float): Mask loss.
        """
        # Original forward pass
        out, M_loss = super(PermutationImportanceTabNet, self).forward(X, y)

        # Apply permutation with a certain probability
        if torch.rand(1).item() < self.permutation_prob:
            # Iterate over each feature to assess its importance
            for i in range(X.size(1)):
                # Clone the input to avoid in-place modifications
                X_permuted = X.clone()

                # Permute the values of the i-th feature across the batch
                X_permuted[:, i] = X_permuted[torch.randperm(X_permuted.size(0)), i]

                # Forward pass with permuted feature
                out_permuted, _ = super(PermutationImportanceTabNet, self).forward(X_permuted, y)

                # Compute predictions
                preds = out.argmax(dim=1)
                preds_permuted = out_permuted.argmax(dim=1)

                # Calculate accuracy
                acc = accuracy_score(y.cpu().numpy(), preds.cpu().numpy())
                acc_perm = accuracy_score(y.cpu().numpy(), preds_permuted.cpu().numpy())

                # Drop in accuracy signifies feature importance
                drop = acc - acc_perm

                # Update importance scores with decay
                self.importance_scores[i] = self.importance_decay * self.importance_scores[i] + (1 - self.importance_decay) * drop

            # Normalize importance scores to sum to 1 for interpretability
            if self.importance_scores.sum() != 0:
                self.importance_scores = self.importance_scores / self.importance_scores.sum()

            # Print feature importance scores
            print("\nFeature Importance Scores after Permutation:")
            for idx, score in enumerate(self.importance_scores):
                feature_name = self.feature_names[idx]
                print(f"{feature_name}: {score.item():.4f}")

        return out, M_loss

# Load the dataset
data = pd.read_csv('/content/fetal_health.csv')

# Display the first five rows to verify
print("First five rows of the dataset:")
print(data.head())

# Check the shape of the dataset
print(f"\nDataset Shape: {data.shape}")

# Features to drop based on prior analysis
features_to_drop = [
    'fetal_movement',
    'histogram_width',
    'histogram_max',
    'mean_value_of_long_term_variability',
    'histogram_number_of_peaks',
    'light_decelerations',
    'histogram_tendency',
    'histogram_number_of_zeroes',
    'severe_decelerations',
    'baseline value',
    'histogram_min'
]

# Drop the specified features
data_dropped = data.drop(columns=features_to_drop)

# Verify the remaining features
print("\nFeatures after dropping less important ones:")
print(data_dropped.columns.tolist())

# Check the new shape of the dataset
print(f"\nNew Dataset Shape after dropping features: {data_dropped.shape}")

# Convert 'fetal_health' to integer
data_dropped['fetal_health'] = data_dropped['fetal_health'].astype(int)

# Mapping numerical classes to descriptive labels
health_mapping = {1: 'Normal', 2: 'Suspect', 3: 'Pathological'}
data_dropped['fetal_health_label'] = data_dropped['fetal_health'].map(health_mapping)

# Display the mapping
print("\nDataset with Mapped Labels:")
print(data_dropped[['fetal_health', 'fetal_health_label']].head())

# Features and target
X = data_dropped.drop(['fetal_health', 'fetal_health_label'], axis=1)
y = data_dropped['fetal_health']

# Initialize SMOTE with 'auto' strategy to resample all classes
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the dataset
X_smote, y_smote = smote.fit_resample(X, y)

# Initialize Tomek Links
tomek = TomekLinks()

# Apply Tomek Links to clean the dataset
X_resampled, y_resampled = tomek.fit_resample(X_smote, y_smote)

# Display the shape of the resampled dataset and class distribution
print(f"\nResampled X shape after SMOTE + Tomek Links: {X_resampled.shape}")
print(f"Resampled y distribution after SMOTE + Tomek Links:\n{y_resampled.value_counts()}")

# Split the resampled data (70% train, 30% test) with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

# Display the shapes of the training and testing sets
print(f"\nTraining set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data
X_test_scaled = scaler.transform(X_test)

# Convert the scaled arrays back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

# Verify scaling by checking min and max values
print("\nMin of Scaled Training Features (Should be 0):")
print(X_train_scaled.min())

print("\nMax of Scaled Training Features (Should be 1):")
print(X_train_scaled.max())

# Adjust the target values so they start from 0
y_train = y_train - 1
y_test = y_test - 1

# Display the adjusted target distributions
print("\nAdjusted y_train distribution:")
print(pd.Series(y_train).value_counts())

print("\nAdjusted y_test distribution:")
print(pd.Series(y_test).value_counts())

# Further split the training data into training and validation sets
X_train_final, X_valid, y_train_final, y_valid = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Display the shapes of the final training and validation sets
print(f"\nFinal Training set shape: {X_train_final.shape}")
print(f"Validation set shape: {X_valid.shape}")

# -------------------
# Hyperparameter Optimization with Optuna
# -------------------
def objective(trial: Trial):
    # Define the hyperparameter space
    n_d = trial.suggest_int('n_d', 32, 128)
    n_a = trial.suggest_int('n_a', 32, 128)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-4, 1e-2, log=True)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [128, 256, 512])

    # Initialize TabNet with current hyperparameters
    tabnet = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        mask_type='sparsemax',
        verbose=0
    )

    # Train the model on the final training set
    tabnet.fit(
        X_train=X_train_final.values,
        y_train=y_train_final.values,
        eval_set=[(X_valid.values, y_valid.values)],
        eval_name=['valid'],
        eval_metric=['accuracy'],
        max_epochs=100,
        patience=20,
        batch_size=batch_size,
        virtual_batch_size=128
    )

    # Predict on the validation set
    y_pred = tabnet.predict(X_valid.values)
    accuracy = accuracy_score(y_valid, y_pred)

    return accuracy

# Create and optimize the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=3600)  # Adjust n_trials and timeout as needed

print("Best Hyperparameters: ", study.best_params)
print("Best Validation Accuracy: ", study.best_value)



ERROR: Could not find a version that satisfies the requirement scikit-learn-contrib (from versions: none)
ERROR: No matching distribution found for scikit-learn-contrib


[I 2025-01-12 07:05:55,243] A new study created in memory with name: no-name-739949e5-f554-43ec-8096-7d7fbfac204c


First five rows of the dataset:
   baseline value  accelerations  fetal_movement  uterine_contractions  \
0           120.0          0.000             0.0                 0.000   
1           132.0          0.006             0.0                 0.006   
2           133.0          0.003             0.0                 0.008   
3           134.0          0.003             0.0                 0.008   
4           132.0          0.007             0.0                 0.008   

   light_decelerations  severe_decelerations  prolongued_decelerations  \
0                0.000                   0.0                       0.0   
1                0.003                   0.0                       0.0   
2                0.003                   0.0                       0.0   
3                0.003                   0.0                       0.0   
4                0.000                   0.0                       0.0   

   abnormal_short_term_variability  mean_value_of_short_term_variability  \
0 

[I 2025-01-12 07:09:08,148] Trial 0 finished with value: 0.950937950937951 and parameters: {'n_d': 128, 'n_a': 67, 'n_steps': 7, 'gamma': 1.6614778909407883, 'lambda_sparse': 0.0003674811482794505, 'learning_rate': 0.053065035211493534, 'batch_size': 256}. Best is trial 0 with value: 0.950937950937951.



Early stopping occurred at epoch 75 with best_epoch = 55 and best_valid_accuracy = 0.96825


...


Early stopping occurred at epoch 93 with best_epoch = 73 and best_valid_accuracy = 0.96248


[I 2025-01-12 08:05:21,911] Trial 22 finished with value: 0.9624819624819625 and parameters: {'n_d': 84, 'n_a': 127, 'n_steps': 7, 'gamma': 1.4214468942829412, 'lambda_sparse': 0.0002334756135464357, 'learning_rate': 0.0014508600133250587, 'batch_size': 128}. Best is trial 13 with value: 0.9711399711399712.



Early stopping occurred at epoch 62 with best_epoch = 42 and best_valid_accuracy = 0.96825


[I 2025-01-12 08:06:55,163] Trial 23 finished with value: 0.9682539682539683 and parameters: {'n_d': 60, 'n_a': 120, 'n_steps': 4, 'gamma': 1.28929895439721, 'lambda_sparse': 0.00014294646624166726, 'learning_rate': 0.0022519727233780364, 'batch_size': 128}. Best is trial 13 with value: 0.9711399711399712.


Best Hyperparameters:  {'n_d': 33, 'n_a': 128, 'n_steps': 3, 'gamma': 1.1761592252229112, 'lambda_sparse': 0.00036511785702883487, 'learning_rate': 0.002956299320724345, 'batch_size': 128}
Best Validation Accuracy:  0.9711399711399712


In [4]:
# 8. Retrain TabNet with Best Hyperparameters Using PermutationImportanceTabNet
# -------------------
# Extract best hyperparameters
best_params = study.best_params

# Adjust keys if necessary
# (Ensure that 'learning_rate' and 'batch_size' are correctly handled)
# In this case, no adjustment is needed as keys are consistent

# Define feature names for interpretability
feature_names = X.columns.tolist()

# Determine the input dimension from the training data
input_dim = X_train_final.shape[1]

# Initialize the Permutation Importance TabNet with the correct input_dim and feature_names
perm_importance_tabnet = PermutationImportanceTabNet(
    input_dim=input_dim,
    feature_names=feature_names,
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    mask_type='sparsemax',
    permutation_prob=0.1,          # 10% chance to apply permutation
    importance_decay=0.99,         # Decay factor for smoothing
    verbose=1
)

# Train the Permutation Importance TabNet model
perm_importance_tabnet.fit(
    X_train=X_train_final.values,
    y_train=y_train_final.values,
    eval_set=[(X_valid.values, y_valid.values), (X_test_scaled.values, y_test.values)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=20,
    batch_size=best_params['batch_size'],
    virtual_batch_size=128
)

# Predict and evaluate on the test set
y_pred_perm_importance = perm_importance_tabnet.predict(X_test_scaled.values)
print("\nPermutation Importance TabNet Classification Report:")
print(classification_report(y_test, y_pred_perm_importance, target_names=['Normal', 'Suspect', 'Pathological']))

# Access and print feature importance scores
print("\nFeature Importance Scores:")
for idx, score in enumerate(perm_importance_tabnet.importance_scores):
    feature_name = perm_importance_tabnet.feature_names[idx]
    print(f"{feature_name}: {score.item():.4f}")

epoch 0  | loss: 0.73588 | train_accuracy: 0.34632 | valid_accuracy: 0.35243 |  0:00:03s
epoch 1  | loss: 0.36865 | train_accuracy: 0.37807 | valid_accuracy: 0.36927 |  0:00:08s
epoch 2  | loss: 0.29494 | train_accuracy: 0.5267  | valid_accuracy: 0.51011 |  0:00:15s
epoch 3  | loss: 0.26794 | train_accuracy: 0.57576 | valid_accuracy: 0.58288 |  0:00:20s
epoch 4  | loss: 0.24623 | train_accuracy: 0.56999 | valid_accuracy: 0.56469 |  0:00:24s
epoch 5  | loss: 0.24563 | train_accuracy: 0.63203 | valid_accuracy: 0.65499 |  0:00:29s
epoch 6  | loss: 0.23157 | train_accuracy: 0.65945 | valid_accuracy: 0.66509 |  0:00:32s
epoch 7  | loss: 0.20214 | train_accuracy: 0.68398 | valid_accuracy: 0.69879 |  0:00:33s
epoch 8  | loss: 0.21204 | train_accuracy: 0.7215  | valid_accuracy: 0.73248 |  0:00:35s
epoch 9  | loss: 0.20335 | train_accuracy: 0.73737 | valid_accuracy: 0.7473  |  0:00:37s
epoch 10 | loss: 0.19136 | train_accuracy: 0.79654 | valid_accuracy: 0.78976 |  0:00:40s
epoch 11 | loss: 0.16